In [1]:
import urllib.request
from urllib.request import Request, urlopen

import requests
import json
import jsonpath

import jsonlines

import pandas as pd

In [ ]:
'''
# request url

#url = "https://api-samenmeten.rivm.nl/v1.0/Things"
#response = requests.get(url)
#html = response.text

response_body = urlopen('https://api-samenmeten.rivm.nl/v1.0/Things')
raw_data = response_body.read()

# creat columns
data = json.loads(raw_data)
a = "name"
station_name = jsonpath.jsonpath(data, "$..name")
station_id = jsonpath.jsonpath(data,"@iot.selfLink")

print (station_name)
#station_coordinates = jsonpath.jsonpath(data,'$..coordinates')
'''

In [4]:
# request url
response_body = urlopen('https://api-samenmeten.rivm.nl/v1.0/Things')
raw_data = response_body.read()
str_data = bytes.decode(raw_data)
json_data = json.loads(str_data)

print(type(json_data))
print(json_data.keys())

<class 'dict'>
dict_keys(['@iot.nextLink', 'value'])


In [5]:
station_id = pd.json_normalize(json_data, record_path=['value'])[['@iot.id']]
station_name = pd.json_normalize(json_data, record_path=['value'])[['name']]
station_owner = pd.json_normalize(json_data,  record_path=['value'])[['properties.owner']]
station_Datastreamslink = pd.json_normalize(json_data,  record_path=['value'])[['Datastreams@iot.navigationLink']]
print (station_id)
print (station_name)
print (station_owner)
print (station_Datastreamslink)

     @iot.id
0       6529
1       6528
2       6527
3       6526
4       6525
..       ...
195     6334
196     6333
197     6332
198     6331
199     6330

[200 rows x 1 columns]
                    name
0    LUC_354724647885109
1              LTD_59393
2              LTD_59389
3              LTD_59387
4              LTD_59375
..                   ...
195  LUC_355523767670583
196  LUC_355523767670443
197  LUC_355523767670427
198  LUC_355523767670385
199  LUC_355523767666409

[200 rows x 1 columns]
       properties.owner
0    Gemeente Rotterdam
1             Luftdaten
2             Luftdaten
3             Luftdaten
4             Luftdaten
..                  ...
195  Gemeente Rotterdam
196  Gemeente Rotterdam
197  Gemeente Rotterdam
198  Gemeente Rotterdam
199  Gemeente Rotterdam

[200 rows x 1 columns]
                        Datastreams@iot.navigationLink
0    https://api-samenmeten.rivm.nl/v1.0/Things(652...
1    https://api-samenmeten.rivm.nl/v1.0/Things(652...
2    https://api-sa

In [6]:
a = 0
b = 0
location_link = pd.json_normalize(json_data, record_path = ['value'])[['Locations@iot.navigationLink']]
station_lat2 = pd.DataFrame(columns = ['latitude'])
station_lon2 = pd.DataFrame(columns = ['longitude'])

while a < len(location_link):
    #step1：get url
    url_location = str(location_link.loc[a,'Locations@iot.navigationLink'])

    #step2：transfer url to dict
    response_location = urlopen (url_location)
    raw_location = response_location.read()
    str_location = bytes.decode(raw_location)
    json_location = json.loads(str_location)

    #step3：get target value
    station_coordinates = pd.json_normalize(json_location, record_path=['value','location','coordinates'])
    station_lat = station_coordinates.loc[0,0]
    station_lon = station_coordinates.loc[1,0]

    #step4：put the value into dataframe
    station_lat2.loc[b,'latitude'] = station_lat
    station_lon2.loc[b,'longitude'] = station_lon

    a = a + 1
    b = b + 1

    #step5：repeat
print (station_lat2)
print (station_lon2)

    latitude
0       4.48
1      4.864
2      5.193
3      5.426
4      5.728
..       ...
195    5.149
196        0
197        0
198        0
199        0

[200 rows x 1 columns]
    longitude
0      51.917
1      51.652
2      52.233
3      52.126
4      53.216
..        ...
195    52.346
196         0
197         0
198         0
199         0

[200 rows x 1 columns]


In [7]:
Joint1 = pd.concat([station_id, station_name, station_owner, station_lat2, station_lon2, station_Datastreamslink], axis=1)
Joint1.rename(columns = {'Datastreams@iot.navigationLink' : '@iot.selfLink'}, inplace = True)
Joint1.set_index('@iot.selfLink',inplace = True)
print (Joint1)

                                                    @iot.id  \
@iot.selfLink                                                 
https://api-samenmeten.rivm.nl/v1.0/Things(6529...     6529   
https://api-samenmeten.rivm.nl/v1.0/Things(6528...     6528   
https://api-samenmeten.rivm.nl/v1.0/Things(6527...     6527   
https://api-samenmeten.rivm.nl/v1.0/Things(6526...     6526   
https://api-samenmeten.rivm.nl/v1.0/Things(6525...     6525   
...                                                     ...   
https://api-samenmeten.rivm.nl/v1.0/Things(6334...     6334   
https://api-samenmeten.rivm.nl/v1.0/Things(6333...     6333   
https://api-samenmeten.rivm.nl/v1.0/Things(6332...     6332   
https://api-samenmeten.rivm.nl/v1.0/Things(6331...     6331   
https://api-samenmeten.rivm.nl/v1.0/Things(6330...     6330   

                                                                   name  \
@iot.selfLink                                                             
https://api-samenmeten.rivm.nl

In [8]:
a = 0
b = 0
station_ave_result = pd.DataFrame(columns = ['average_value'])
Joint2 = pd.DataFrame(columns = ['@iot.selfLink', 'description', 'average_value', 'unitOfMeasurement.symbol'])
datastreams_link = pd.json_normalize(json_data, record_path = ['value'])[['Datastreams@iot.navigationLink']]

while a < len (datastreams_link):
    url_datastreams = str(datastreams_link.loc[a,'Datastreams@iot.navigationLink'])
    response_datastreams = urlopen (url_datastreams)
    raw_datastreams = response_datastreams.read()
    str_datastreams = bytes.decode(raw_datastreams)
    json_datastreams = json.loads(str_datastreams)
    
    station_symbols = pd.json_normalize(json_datastreams,  record_path = ['value'])[['unitOfMeasurement.symbol']]
    station_descriptions = pd.json_normalize(json_datastreams,  record_path = ['value'])[['description']]
    station_datastream_selflinks = pd.json_normalize(json_datastreams,  record_path = ['value'])[['@iot.selfLink']]
    station_datastream_selflinks.iloc[:,0] = url_datastreams
    
#    print (datastreams_link)
#    print (station_descriptions)
#    print (station_datastream_selflinks)
    
    
    obeservations_link = pd.json_normalize(json_datastreams, record_path = ['value'])[['Observations@iot.navigationLink']]
    
    while b < len (obeservations_link):
        url_obeservations = str(obeservations_link.loc[b,'Observations@iot.navigationLink'])
        response_obeservations = urlopen (url_obeservations)
        raw_obeservations = response_obeservations.read()
        str_obeservations = bytes.decode(raw_obeservations)
        json_obeservations = json.loads(str_obeservations)
        
        try:
            station_resulttime = pd.json_normalize(json_obeservations,  record_path = ['value'])[['phenomenonTime']]
        except:
            station_ave_result.loc[b,'average_value'] =  None
        else:
            station_results = pd.json_normalize(json_obeservations,  record_path = ['value'])[['result']]
            station_ave_result.loc[b,'average_value'] = station_results['result'].mean()
            
        b = b + 1
        
    Combination = pd.concat([station_datastream_selflinks, station_descriptions, station_ave_result, station_symbols], axis=1)
    Joint2 = pd.concat([Joint2, Combination], ignore_index = True)
    
    a = a + 1
    b = 0
    station_ave_result = pd.DataFrame(columns = ['average_value'])


print (len (obeservations_link))
print (station_resulttime)
print (Joint2)


5
              phenomenonTime
0   2022-02-16T22:00:00.000Z
1   2022-02-16T21:00:00.000Z
2   2022-02-16T20:00:00.000Z
3   2022-02-16T19:00:00.000Z
4   2022-02-16T18:00:00.000Z
5   2022-02-16T17:00:00.000Z
6   2022-02-16T16:00:00.000Z
7   2022-02-16T15:00:00.000Z
8   2022-02-16T14:00:00.000Z
9   2022-02-16T13:00:00.000Z
10  2022-02-16T12:00:00.000Z
11  2022-02-16T11:00:00.000Z
12  2022-02-16T10:00:00.000Z
13  2022-02-16T09:00:00.000Z
14  2022-02-16T08:00:00.000Z
15  2022-02-16T07:00:00.000Z
16  2022-02-16T06:00:00.000Z
17  2022-02-16T05:00:00.000Z
                                          @iot.selfLink  \
0     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
1     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
2     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
3     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
4     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
...                                                 ...   
1066  https://api-samenmeten.rivm.

In [9]:
result = Joint2.join(Joint1, on='@iot.selfLink')
print (result)
result.to_excel('open_air_pollution.xlsx')

                                          @iot.selfLink  \
0     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
1     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
2     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
3     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
4     https://api-samenmeten.rivm.nl/v1.0/Things(652...   
...                                                 ...   
1066  https://api-samenmeten.rivm.nl/v1.0/Things(633...   
1067  https://api-samenmeten.rivm.nl/v1.0/Things(633...   
1068  https://api-samenmeten.rivm.nl/v1.0/Things(633...   
1069  https://api-samenmeten.rivm.nl/v1.0/Things(633...   
1070  https://api-samenmeten.rivm.nl/v1.0/Things(633...   

                          description average_value unitOfMeasurement.symbol  \
0          LUC_354724647885109-4-pres    998.508824                      hPa   
1            LUC_354724647885109-4-rh     39.131176                        %   
2          LUC_354724647885109-4-temp     16.011765